# Rag From Scratch: Routing

Logic + Semantica Routing

We decompose a question to the right source

<img src="../../data/images/routing.png"  />

Imports and env variables

In [40]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [41]:
hugging_face_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
langchain_token = os.getenv("LANGCHAIN_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_env = os.getenv("PINECONE_ENV")
open_api_key = os.getenv("OPENAI_API_KEY")

In [42]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from tqdm import tqdm
import tiktoken
import numpy as np
from typing import Literal
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# Part 10: Logical and Semantic routing

### Logical Routing

It works like a classification, basically you give the llm a few "labels" and based on the question you ask it will output a specific answer/label and it will select the function tied to that label. We bind the function calling to the LLM

<img src="../../data/images/routing_logic_semantic.png"  />

Data Model

In [43]:
# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

LLM with function call

In [44]:
from langchain_openai import ChatOpenAI

# LLM with function call 
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

In [45]:
# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

We used function calling to produce structured output.

<img src="../../data/images/function_calling.png"  />

In [ ]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

In [ ]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for js_docs"
    else:
        ### Logic here 
        return "golang_docs"

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

In [ ]:
full_chain.invoke({"question": question})

### Semantic Routing

<img src="../../data/images/semantic_routing.png"  />

In [47]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [48]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceEndpoint(repo_id=repo_id,
                          huggingfacehub_api_token=hugging_face_token,
                          temperature=0.1)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\Hori\.cache\huggingface\token
Login successful


Prompts

In [49]:
# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

Embed prompts

In [51]:
from langchain_huggingface import HuggingFaceEmbeddings

# Embed prompts
embeddings = HuggingFaceEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

Route question to prompt 

In [52]:
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)

Chain

In [55]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)

In [56]:
print(chain.invoke("What's a black hole"))

Using PHYSICS
?

Answer:
A black hole is a region in space where the gravitational pull is so strong that nothing, not even light, can escape. It's called a "black" hole because it absorbs all the light that hits it, making it appear black. Black holes are formed when a massive star collapses in on itself at the end of its life. The exact nature of what happens inside a black hole is still a mystery and is a topic of ongoing research in physics.


In [57]:
print(chain.invoke("What's arithmetics"))

Using MATH
?

To answer this question, let's break it down into component parts:

1. What is the meaning of the word "arithmetic"?
2. What is the field of study that deals with arithmetic?

Answer to component part 1:
The word "arithmetic" comes from the Greek word "arithmetikos," which means "relating to numbers." So, arithmetic is a branch of mathematics that deals with the properties and manipulation of numbers.

Answer to component part 2:
Arithmetic is a subfield of mathematics that deals with the properties and manipulation of numbers. It includes the study of addition, subtraction, multiplication, division, and the relationships between numbers. Arithmetic is often considered the foundation of mathematics, as it provides the building blocks for more advanced mathematical concepts.

Putting it all together:
Arithmetic is a branch of mathematics that deals with the properties and manipulation of numbers. It includes the study of addition, subtraction, multiplication, division, and